In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
from pylab import rcParams
%pylab inline
rcParams['figure.figsize'] = 12, 6
from datetime import datetime
import re
import math
from sklearn.metrics import make_scorer
from sklearn import preprocessing
import xgboost as xgb
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.grid_search import GridSearchCV

Populating the interactive namespace from numpy and matplotlib


In [2]:
# d = pd.read_csv('tmp/engineered_features.csv')

In [3]:
# del d['Date']
# del d['PromoInterval']
# del d['Customers']
# d = pd.get_dummies(d, ['StateHoliday', 'StoreType', 'Assortment'])
# d = d.fillna(-1)

In [4]:
# sales = d.Sales
# del d['Sales']
# d = DataFrame(preprocessing.scale(d), columns=d.columns)
# d['Sales'] = sales
# d.describe().T[:5]

In [5]:
# d.to_csv('tmp/scaled_features.csv')
# del d

In [6]:
def rmspe_func(y_true, y_pred, **kwargs):
    if len(y_pred) != len(y_true):
        raise Exception("different length series passed to scorer function")
    d = DataFrame({'yt': y_true, 'yp': y_pred})
    d = d[d.yt != 0]
    return np.sqrt((1.0/len(d.yt))*(np.square(((d.yt - d.yp)/d.yt)).sum()))
print rmspe_func(Series([2.0, 2.0, 0.0, 0.0]), Series([1.0, 5.0, 100.0, 0.0]))
print rmspe_func(Series([2.0, 2.0, 0.0, 0.0]), Series([0.0, 0.0, 0.0, 0.0]))
rmspe = make_scorer(rmspe_func, greater_is_better=False)

1.11803398875
1.0


In [7]:
d = pd.read_csv('tmp/scaled_features.csv')
d = d[d.Sales > 0]
d = d.iloc[np.random.permutation(len(d))]
lim_tr = int(0.4 * len(d))
tr = d[:lim_tr]
trl = tr.Sales
del tr['Sales']
lim_ho = int(0.5 * len(d))
ho = d[lim_tr:lim_ho]
hol = ho.Sales
del ho['Sales']
del d

In [8]:
def eval_pred(p):
    return rmspe_func(hol, p)
print eval_pred(zeros_like(hol))

1.0


In [9]:
def eval_model(m):
    m.fit(tr, trl)
    p = m.predict(ho)
    return eval_pred(p)

In [10]:
# param_grid = {
#     'n_estimators': [200],
#     'nthread': [6],
#     'learning_rate': [0.15],
#     'max_depth': [15, 25],
#     'objective':['reg:linear'],
#     'min_child_weight': [5, 10]
# }
# m = GridSearchCV(xgb.XGBRegressor(), param_grid, cv=2, scoring=rmspe)
# print eval_model(m)
# print m.best_params_
# #best result: ~0.15

In [ ]:
param_grid = {
    'n_neighbors': [1, 7, 50, 350],
    'weights': ['uniform', 'distance'],
    'p': [1, 2, 3]
}
m = GridSearchCV(KNeighborsRegressor(), param_grid, cv=2, scoring=rmspe)
print eval_model(m)
print m.best_params_

0.502272698292
{'n_neighbors': 50, 'weights': 'uniform', 'p': 1}


In [ ]:
# param_grid = {
#     'C': [0.5, 1.0, 2.0],
#     'epsilon': [0.01, 0.1, 0.5],
#     'kernel': ['rbf', 'linear', 'poly']
# }
param_grid = {
    'C': [1.0],
    'epsilon': [0.1],
    'kernel': ['linear']
}
m = GridSearchCV(SVR(), param_grid, cv=2, scoring=rmspe)
print eval_model(m)
print m.best_params_